This is an example of merging precipitation data with the PFAS file. The precipitation data is for Orange County, 2010-2019 and was accessed from the NOAA Climate Data Online Search (https://www.ncdc.noaa.gov/cdo-web/search?datasetid=GHCND)

In [215]:
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns
import math

def deg2rad(deg):
    return(deg*(math.pi/180.))
def getDistanceFromLatLonInKm(lat1,lon1,lat2,lon2):
    R=6371.
    dlat=deg2rad(lat2-lat1)
    dlon=deg2rad(lon2-lon1)
    a=math.sin(dlat/2.)*math.sin(dlat/2.) + math.cos(deg2rad(lat1)) *\
    math.cos(deg2rad(lat2)) *math.sin(dlon/2.)*math.sin(dlon/2.)

    b = 2. * math.atan2(math.sqrt(a),math.sqrt(1-a))
    d=R*b
    return(d)

prec=pd.read_csv("precip_2010_2019.csv")[['STATION','LATITUDE','LONGITUDE','ELEVATION','DATE','PRCP','TAVG','TMAX','TMIN']]
prec['DATE']=pd.to_datetime(prec['DATE'])
dat=pd.read_excel("SAR-Imperial_537Data_AsOf08-08-2019.xlsx",sheet_name='All')
location_data=pd.read_excel("SAR-Imperial_537Data_AsOf08-08-2019.xlsx",sheet_name='SAR-IMPERIAL-01 Location')
dat['latitude']=float(location_data['Latitude_WGS84'])
dat['longitude']=float(location_data['Longitude_WGS84'])

In [159]:
#Function to take precipitation data and match it with the location of the PFAS data
def match_prec_pfas(prec_table,pfas_table,
                    prec_stat_name='STATION',pfas_stat_name='Station Name',
                   prec_lat_name='LATITUDE',prec_lon_name='LONGITUDE',
                   pfas_lat_name='latitude',pfas_lon_name='longitude'):
    #Find the closest precip station to the PFAS station
    prec_stations=prec_table[[prec_stat_name,prec_lat_name,prec_lon_name]].drop_duplicates()
    pfas_stations=pfas_table[[pfas_stat_name,pfas_lat_name,pfas_lon_name]].drop_duplicates()
    #Make table of distances
    comb_dist=pd.DataFrame()
    for s in pfas_stations[pfas_stat_name]:
        #Get the lat/lon coords for the station
        pfas_sub=pfas_stations[pfas_stations[pfas_stat_name]==s]
        pfas_lat=float(pfas_sub[pfas_lat_name])
        pfas_lon=float(pfas_sub[pfas_lon_name])
        for p in prec_stations[prec_stat_name]:
            prec_sub=prec_stations[prec_stations[prec_stat_name]==p]
            prec_lat=float(prec_sub[prec_lat_name])
            prec_lon=float(prec_sub[prec_lon_name])
            dict_store={}
            dict_store['pfas_s']=s
            dict_store['prec_s']=p
            dict_store['dist_betw']=getDistanceFromLatLonInKm(pfas_lat,pfas_lon,prec_lat,prec_lon)
            comb_dist=comb_dist.append(dict_store,ignore_index=True)
    #Find the min distance
    stat_match=pd.DataFrame(columns=['pfas_s','prec_s','dist_betw'])
    for s in pfas_stations[pfas_stat_name]:
        comb_sub=comb_dist[comb_dist['pfas_s']==s]
        min_dist=comb_sub.loc[comb_sub['dist_betw'].idxmin()]
        stat_match=stat_match.append(min_dist)
    return(stat_match)


In [148]:
min_dist=match_prec_pfas(prec,dat)

In [89]:
#Merge the data by the date?
prec_tomerge=prec[prec['STATION']==str(min_dist['prec_s'])]
dat_merged=pd.merge(prec_tomerge,dat,left_on='DATE',right_on='Sample Date')

In [149]:
dat_sort=dat_merged.sort_values(by=['DATE'])
dat_sort.to_csv('SAR-Imperial_merged_precip.csv')

In [216]:
#Another dataset
new_dat=pd.read_csv('gama_pf_orange.csv')
new_dat['DATE']=pd.to_datetime(new_dat['DATE'])

In [217]:
dat_match=match_prec_pfas(prec,new_dat,pfas_stat_name='WELL ID',pfas_lat_name='APPROXIMATE LATITUDE',pfas_lon_name='APPROXIMATE LONGITUDE')

In [218]:
oc_merged=pd.DataFrame()
for w in new_dat['WELL ID'].unique():
    dat_sub=new_dat[new_dat['WELL ID']==w]
    match_sub=dat_match[dat_match['pfas_s']==w]
    prec_sub=prec[prec['STATION'].str.match(match_sub['prec_s'].item())]
    merged_dat=pd.merge(dat_sub,prec_sub,left_on='DATE',right_on='DATE')
    print(merged_dat)
    oc_merged=oc_merged.append(merged_dat)

        WELL ID  RESULTS CHEMICAL       DATE UNITS QUALIFER   RL  \
0   3010027-023   0.0044     PFOA 2019-04-23  UG/L      NaN  UNK   
1   3010027-023      NaN    PFHPA 2019-04-23   NaN      NaN  UNK   
2   3010027-023      NaN     PFHA 2019-04-23   NaN      NaN  UNK   
3   3010027-023      NaN   PFTEDA 2019-04-23   NaN      NaN  UNK   
4   3010027-023      NaN  PFUNDCA 2019-04-23   NaN      NaN  UNK   
5   3010027-023      NaN     PFNA 2019-04-23   NaN      NaN  UNK   
6   3010027-023      NaN     PFOS 2019-04-23   NaN      NaN  UNK   
7   3010027-023      NaN    PFDOA 2019-04-23   NaN      NaN  UNK   
8   3010027-023      NaN    PFBSA 2019-04-23   NaN      NaN  UNK   
9   3010027-023      NaN   PFHXSA 2019-04-23   NaN      NaN  UNK   
10  3010027-023      NaN   PFNDCA 2019-04-23   NaN      NaN  UNK   
11  3010027-023      NaN  PFTRIDA 2019-04-23   NaN      NaN  UNK   

    APPROXIMATE LATITUDE  APPROXIMATE LONGITUDE  WELL TYPE  ...  OTHER NAMES  \
0              33.786491           

        WELL ID  RESULTS CHEMICAL       DATE UNITS QUALIFER   RL  \
0   3010068-001      NaN  PFTRIDA 2019-04-23   NaN      NaN  UNK   
1   3010068-001      NaN  PFUNDCA 2019-04-23   NaN      NaN  UNK   
2   3010068-001      NaN     PFHA 2019-04-23   NaN      NaN  UNK   
3   3010068-001      NaN    PFHPA 2019-04-23   NaN      NaN  UNK   
4   3010068-001      NaN   PFHXSA 2019-04-23   NaN      NaN  UNK   
5   3010068-001      NaN     PFOS 2019-04-23   NaN      NaN  UNK   
6   3010068-001      NaN     PFNA 2019-04-23   NaN      NaN  UNK   
7   3010068-001      NaN    PFBSA 2019-04-23   NaN      NaN  UNK   
8   3010068-001      NaN   PFTEDA 2019-04-23   NaN      NaN  UNK   
9   3010068-001      NaN   PFNDCA 2019-04-23   NaN      NaN  UNK   
10  3010068-001      NaN    PFDOA 2019-04-23   NaN      NaN  UNK   
11  3010068-001   0.0219     PFOA 2019-04-23  UG/L      NaN  UNK   

    APPROXIMATE LATITUDE  APPROXIMATE LONGITUDE  WELL TYPE  ...  OTHER NAMES  \
0              33.786491           

        WELL ID  RESULTS CHEMICAL       DATE UNITS QUALIFER   RL  \
0   3010027-008      NaN   PFTEDA 2019-04-23   NaN      NaN  UNK   
1   3010027-008   0.0201     PFOA 2019-04-23  UG/L      NaN  UNK   
2   3010027-008      NaN     PFOS 2019-04-23   NaN      NaN  UNK   
3   3010027-008      NaN    PFBSA 2019-04-23   NaN      NaN  UNK   
4   3010027-008      NaN   PFNDCA 2019-04-23   NaN      NaN  UNK   
5   3010027-008      NaN    PFDOA 2019-04-23   NaN      NaN  UNK   
6   3010027-008      NaN     PFHA 2019-04-23   NaN      NaN  UNK   
7   3010027-008      NaN    PFHPA 2019-04-23   NaN      NaN  UNK   
8   3010027-008      NaN  PFUNDCA 2019-04-23   NaN      NaN  UNK   
9   3010027-008      NaN   PFHXSA 2019-04-23   NaN      NaN  UNK   
10  3010027-008      NaN     PFNA 2019-04-23   NaN      NaN  UNK   
11  3010027-008      NaN  PFTRIDA 2019-04-23   NaN      NaN  UNK   

    APPROXIMATE LATITUDE  APPROXIMATE LONGITUDE  WELL TYPE  ...  OTHER NAMES  \
0              33.815464           

        WELL ID  RESULTS CHEMICAL       DATE UNITS QUALIFER   RL  \
0   3010037-003   0.0262     PFOA 2018-02-05  UG/L      NaN  UNK   
1   3010037-003      NaN     PFOS 2018-02-05   NaN      NaN  UNK   
2   3010037-003      NaN     PFOS 2018-04-25   NaN      NaN  UNK   
3   3010037-003   0.0239     PFOA 2018-04-25  UG/L      NaN  UNK   
4   3010037-003      NaN  PFTRIDA 2019-04-04   NaN      NaN  UNK   
5   3010037-003      NaN    PFHPA 2019-04-04   NaN      NaN  UNK   
6   3010037-003      NaN  PFUNDCA 2019-04-04   NaN      NaN  UNK   
7   3010037-003      NaN     PFNA 2019-04-04   NaN      NaN  UNK   
8   3010037-003      NaN    PFBSA 2019-04-04   NaN      NaN  UNK   
9   3010037-003      NaN     PFHA 2019-04-04   NaN      NaN  UNK   
10  3010037-003      NaN   PFTEDA 2019-04-04   NaN      NaN  UNK   
11  3010037-003      NaN    PFDOA 2019-04-04   NaN      NaN  UNK   
12  3010037-003   0.0207     PFOA 2019-04-04  UG/L      NaN  UNK   
13  3010037-003      NaN   PFHXSA 2019-04-04   N

        WELL ID  RESULTS CHEMICAL       DATE UNITS QUALIFER   RL  \
0   3010037-023      NaN    PFBSA 2019-05-08   NaN      NaN  UNK   
1   3010037-023      NaN     PFNA 2019-05-08   NaN      NaN  UNK   
2   3010037-023      NaN   PFNDCA 2019-05-08   NaN      NaN  UNK   
3   3010037-023      NaN   PFHXSA 2019-05-08   NaN      NaN  UNK   
4   3010037-023      NaN  PFUNDCA 2019-05-08   NaN      NaN  UNK   
5   3010037-023   0.0210     PFOA 2019-05-08  UG/L      NaN  UNK   
6   3010037-023      NaN     PFHA 2019-05-08   NaN      NaN  UNK   
7   3010037-023      NaN     PFOS 2019-05-08   NaN      NaN  UNK   
8   3010037-023      NaN    PFDOA 2019-05-08   NaN      NaN  UNK   
9   3010037-023      NaN   PFTEDA 2019-05-08   NaN      NaN  UNK   
10  3010037-023      NaN  PFTRIDA 2019-05-08   NaN      NaN  UNK   
11  3010037-023      NaN    PFHPA 2019-05-08   NaN      NaN  UNK   
12  3010037-023      NaN   PFTEDA 2019-04-04   NaN      NaN  UNK   
13  3010037-023      NaN   PFNDCA 2019-04-04   N

...

        WELL ID  RESULTS CHEMICAL       DATE UNITS QUALIFER   RL  \
0   3010001-049      NaN     PFHA 2019-04-09   NaN      NaN  UNK   
1   3010001-049      NaN    PFHPA 2019-04-09   NaN      NaN  UNK   
2   3010001-049      NaN   PFNDCA 2019-04-09   NaN      NaN  UNK   
3   3010001-049      NaN     PFNA 2019-04-09   NaN      NaN  UNK   
4   3010001-049      NaN    PFDOA 2019-04-09   NaN      NaN  UNK   
5   3010001-049   0.0048     PFOA 2019-04-09  UG/L      NaN  UNK   
6   3010001-049      NaN  PFTRIDA 2019-04-09   NaN      NaN  UNK   
7   3010001-049      NaN    PFBSA 2019-04-09   NaN      NaN  UNK   
8   3010001-049      NaN   PFHXSA 2019-04-09   NaN      NaN  UNK   
9   3010001-049      NaN  PFUNDCA 2019-04-09   NaN      NaN  UNK   
10  3010001-049      NaN     PFOS 2019-04-09   NaN      NaN  UNK   
11  3010001-049      NaN   PFTEDA 2019-04-09   NaN      NaN  UNK   

    APPROXIMATE LATITUDE  APPROXIMATE LONGITUDE  WELL TYPE  ...   OTHER NAMES  \
0              33.844437          

Empty DataFrame
Columns: [WELL ID, RESULTS, CHEMICAL, DATE, UNITS, QUALIFER, RL, APPROXIMATE LATITUDE, APPROXIMATE LONGITUDE, WELL TYPE, WELL DEPTH (FT), TOP OF SCREEN (FT), SCREEN LENGTH (FT), SOURCE, SOURCE NAME, OTHER NAMES, YEAR, STATION, LATITUDE, LONGITUDE, ELEVATION, PRCP, TAVG, TMAX, TMIN]
Index: []

[0 rows x 25 columns]
Empty DataFrame
Columns: [WELL ID, RESULTS, CHEMICAL, DATE, UNITS, QUALIFER, RL, APPROXIMATE LATITUDE, APPROXIMATE LONGITUDE, WELL TYPE, WELL DEPTH (FT), TOP OF SCREEN (FT), SCREEN LENGTH (FT), SOURCE, SOURCE NAME, OTHER NAMES, YEAR, STATION, LATITUDE, LONGITUDE, ELEVATION, PRCP, TAVG, TMAX, TMIN]
Index: []

[0 rows x 25 columns]
Empty DataFrame
Columns: [WELL ID, RESULTS, CHEMICAL, DATE, UNITS, QUALIFER, RL, APPROXIMATE LATITUDE, APPROXIMATE LONGITUDE, WELL TYPE, WELL DEPTH (FT), TOP OF SCREEN (FT), SCREEN LENGTH (FT), SOURCE, SOURCE NAME, OTHER NAMES, YEAR, STATION, LATITUDE, LONGITUDE, ELEVATION, PRCP, TAVG, TMAX, TMIN]
Index: []

[0 rows x 25 columns]
Empt

Empty DataFrame
Columns: [WELL ID, RESULTS, CHEMICAL, DATE, UNITS, QUALIFER, RL, APPROXIMATE LATITUDE, APPROXIMATE LONGITUDE, WELL TYPE, WELL DEPTH (FT), TOP OF SCREEN (FT), SCREEN LENGTH (FT), SOURCE, SOURCE NAME, OTHER NAMES, YEAR, STATION, LATITUDE, LONGITUDE, ELEVATION, PRCP, TAVG, TMAX, TMIN]
Index: []

[0 rows x 25 columns]
Empty DataFrame
Columns: [WELL ID, RESULTS, CHEMICAL, DATE, UNITS, QUALIFER, RL, APPROXIMATE LATITUDE, APPROXIMATE LONGITUDE, WELL TYPE, WELL DEPTH (FT), TOP OF SCREEN (FT), SCREEN LENGTH (FT), SOURCE, SOURCE NAME, OTHER NAMES, YEAR, STATION, LATITUDE, LONGITUDE, ELEVATION, PRCP, TAVG, TMAX, TMIN]
Index: []

[0 rows x 25 columns]
Empty DataFrame
Columns: [WELL ID, RESULTS, CHEMICAL, DATE, UNITS, QUALIFER, RL, APPROXIMATE LATITUDE, APPROXIMATE LONGITUDE, WELL TYPE, WELL DEPTH (FT), TOP OF SCREEN (FT), SCREEN LENGTH (FT), SOURCE, SOURCE NAME, OTHER NAMES, YEAR, STATION, LATITUDE, LONGITUDE, ELEVATION, PRCP, TAVG, TMAX, TMIN]
Index: []

[0 rows x 25 columns]
    

                    WELL ID  RESULTS CHEMICAL       DATE UNITS QUALIFER  RL  \
0   L10003191714-CONDENSATE      NaN     PFHA 2017-08-29   NaN        =  50   
1   L10003191714-CONDENSATE      NaN   PFTEDA 2017-08-29   NaN       ND  50   
2   L10003191714-CONDENSATE      NaN  PFTRIDA 2017-08-29   NaN       ND  50   
3   L10003191714-CONDENSATE     1.20     PFOA 2017-08-29  UG/L        =  50   
4   L10003191714-CONDENSATE     0.16    PFOSA 2017-08-29  UG/L        =  50   
5   L10003191714-CONDENSATE      NaN  PFUNDCA 2017-08-29   NaN       ND  50   
6   L10003191714-CONDENSATE      NaN     PFNA 2017-08-29   NaN        =  50   
7   L10003191714-CONDENSATE      NaN   PFNDCA 2017-08-29   NaN        =  50   
8   L10003191714-CONDENSATE     0.00    PFOSA 2017-08-29  UG/L       ND  50   
9   L10003191714-CONDENSATE      NaN    PFDOA 2017-08-29   NaN       ND  50   
10  L10003191714-CONDENSATE      NaN    PFHPA 2017-08-29   NaN        =  50   

    APPROXIMATE LATITUDE  APPROXIMATE LONGITUDE   W

        WELL ID  RESULTS CHEMICAL       DATE UNITS QUALIFER   RL  \
0   3010001-062   0.0126     PFOA 2019-04-10  UG/L      NaN  UNK   
1   3010001-062      NaN   PFTEDA 2019-04-10   NaN      NaN  UNK   
2   3010001-062      NaN  PFTRIDA 2019-04-10   NaN      NaN  UNK   
3   3010001-062      NaN   PFNDCA 2019-04-10   NaN      NaN  UNK   
4   3010001-062      NaN    PFHPA 2019-04-10   NaN      NaN  UNK   
5   3010001-062      NaN     PFNA 2019-04-10   NaN      NaN  UNK   
6   3010001-062      NaN     PFHA 2019-04-10   NaN      NaN  UNK   
7   3010001-062      NaN    PFBSA 2019-04-10   NaN      NaN  UNK   
8   3010001-062      NaN   PFHXSA 2019-04-10   NaN      NaN  UNK   
9   3010001-062      NaN     PFOS 2019-04-10   NaN      NaN  UNK   
10  3010001-062      NaN  PFUNDCA 2019-04-10   NaN      NaN  UNK   
11  3010001-062      NaN    PFDOA 2019-04-10   NaN      NaN  UNK   

    APPROXIMATE LATITUDE  APPROXIMATE LONGITUDE  WELL TYPE  ...   OTHER NAMES  \
0              33.844437          

In [219]:
oc_merged

,WELL ID,RESULTS,CHEMICAL,DATE,UNITS,QUALIFER,RL,APPROXIMATE LATITUDE,APPROXIMATE LONGITUDE,WELL TYPE,...,OTHER NAMES,YEAR,STATION,LATITUDE,LONGITUDE,ELEVATION,PRCP,TAVG,TMAX,TMIN
0,3010027-023,0.0044,PFOA,2019-04-23,UG/L,NaN,UNK,33.786491,-117.839104,MUNICIPAL,...,WELL 25,2019,US1CAOR0042,33.7655,-117.8878,33.5,0.0,NaN,NaN,NaN
1,3010027-023,NaN,PFHPA,2019-04-23,NaN,NaN,UNK,33.786491,-117.839104,MUNICIPAL,...,WELL 25,2019,US1CAOR0042,33.7655,-117.8878,33.5,0.0,NaN,NaN,NaN
2,3010027-023,NaN,PFHA,2019-04-23,NaN,NaN,UNK,33.786491,-117.839104,MUNICIPAL,...,WELL 25,2019,US1CAOR0042,33.7655,-117.8878,33.5,0.0,NaN,NaN,NaN
3,3010027-023,NaN,PFTEDA,2019-04-23,NaN,NaN,UNK,33.786491,-117.839104,MUNICIPAL,...,WELL 25,2019,US1CAOR0042,33.7655,-117.8878,33.5,0.0,NaN,NaN,NaN
4,3010027-023,NaN,PFUNDCA,2019-04-23,NaN,NaN,UNK,33.786491,-117.839104,MUNICIPAL,...,WELL 25,2019,US1CAOR0042,33.7655,-117.8878,33.5,0.0,NaN,NaN,NaN
5,3010027-023,NaN,PFNA,2019-04-23,NaN,NaN,UNK,33.786491,-117.839104,MUNICIPAL,...,WELL 25,2019,US1CAOR0042,33.7655,-117.8878,33.5,0.0,NaN,NaN,NaN
6,3010027-023,NaN,PFOS,2019-04-23,NaN,NaN,UNK,33.786491,-117.839104,MUNICIPAL,...,WELL 25,2019,US1CAOR0042,33.7655,-117.8878,33.5,0.0,NaN,NaN,NaN
7,3010027-023,NaN,PFDOA,2019-04-23,NaN,NaN,UNK,33.786491,-117.839104,MUNICIPAL,...,WELL 25,2019,US1CAOR0042,33.7655,-117.8878,33.5,0.0,NaN,NaN,NaN
8,3010027-023,NaN,PFBSA,2019-04-23,NaN,NaN,UNK,33.786491,-117.839104,MUNICIPAL,...,WELL 25,2019,US1CAOR0042,33.7655,-117.8878,33.5,0.0,NaN,NaN,NaN
9,3010027-023,NaN,PFHXSA,2019-04-23,NaN,NaN,UNK,33.786491,-117.839104,MUNICIPAL,...,WELL 25,2019,US1CAOR0042,33.7655,-117.8878,33.5,0.0,NaN,NaN,NaN
